# Test the Stereo Encoder

In [4]:
# import stereo network
from im2mesh.encoder.stereo import HdrnAlphaStereo
from typing import Optional, List


In [7]:
@dataclass
class ModelConfig(object):
    model_file: str = None
    model_name: str = None 

    fe_features: int = 16
    fe_internal_features: int = 32
    num_disparities: int = 256
    downsample_factor: int = 8

    checkpoint: Optional[str] = None

model = HdrnAlphaStereo(hparams())

AttributeError: 'hparams' object has no attribute 'fe_features'

In [ ]:
data_dir = '../../prototypes/1a97f3c83016abca21d0de04f408950f/'